In [23]:
import kagglehub
import csv
import pandas as pd
import math
import numpy as np
import pickle
import math
import os
import gradio as gr
from langchain_core.output_parsers import StrOutputParser

from io import BytesIO
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.http import MediaIoBaseDownload

from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory  # Used to store chat history in memory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import OpenAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain.tools import tool
from langchain.tools.retriever import create_retriever_tool
from langchain_community.tools.tavily_search import TavilySearchResults

### Load data and prepare documents

In [5]:
%%time
SCOPES = ['https://www.googleapis.com/auth/drive']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)

# Authenticate if no valid credentials exist
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file('credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    
    # Save the credentials for future use
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

# Initialize Google Drive API service
drive_service = build('drive', 'v3', credentials=creds)

# List files in Google Drive
results = drive_service.files().list(
    q="name='train_100.csv'",  # Query to search by filename
    spaces='drive',
    fields="files(id, name)"
).execute()

files = results.get("files", [])

if not files:
    print("No file found.")
else:
    file_id = files[0]['id']
    print(f"File ID: {file_id}")

    # Request to download the file
request = drive_service.files().get_media(fileId=file_id)
file = BytesIO()
downloader = MediaIoBaseDownload(file, request)

done = False
while not done:
    status, done = downloader.next_chunk()
    print(f"Download {int(status.progress() * 100)}%.")

# Move to the beginning of the file
file.seek(0)

# Read CSV into Pandas DataFrame
df = pd.read_csv(file, index_col=0)

# Display the first few rows
df.head()

product_description = []
product_description_len = []

row_num = 0
# Iterating through each row in the dataframe df2
for row in df.iterrows():
    if row_num % 100000 == 0:
        print(f"processing row {row_num}")
    row_num += 1
    product = ""  # Initialize an empty string to accumulate product details

    # Extracting the product title from the current row
    title = row[1]["TITLE"]

    # Checking if the title is valid (not NaN or missing)
    if type(title) != float or not math.isnan(title):
        product += "Title\n" + title + "\n"  # Append the title to the product description

    # Extracting the product description from the current row
    description = row[1]["DESCRIPTION"]

    # Checking if the description is valid (not NaN or missing)
    if type(description) != float or not math.isnan(description):
        product += "Description\n" + description + "\n"  # Append the description to the product details

    # Check if either title or description was added
    added_content = title or description
    if added_content:
        product = product.strip()  # Remove any leading/trailing whitespace
        product_description.append(product)  # Add the formatted product details to the list
        product_description_len.append(len(product))  # Store the length of the product description

print(f"Number of elements {len(product_description)}")
print("Number of items", len(product_description_len))
print("Min length of the description", np.min(product_description_len))
print("Avg length of the description", np.mean(product_description_len))
print("Max length of the description", np.max(product_description_len))

File ID: 1f1E8hjgOWFEXIRsK5gV-j-XD2gYBwPN2
Download 100%.
processing row 0
Number of elements 100
Number of items 100
Min length of the description 18
Avg length of the description 385.9
Max length of the description 1834
CPU times: user 48.4 ms, sys: 10.2 ms, total: 58.6 ms
Wall time: 1.02 s


### Build retrieval chain

In [6]:
%%time
# Chunking the Data(Product Description)
# See this for more details https://python.langchain.com/v0.1/docs/modules/data_connection/document_transformers/recursive_text_splitter/
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=250,
    chunk_overlap=20,
    length_function=len,
    is_separator_regex=False,
)
documents = text_splitter.create_documents(product_description)

# Create an embedding model using LangChain.
# See https://python.langchain.com/docs/integrations/text_embedding/ for a list of available embedding models on LangChain
embeddings = OpenAIEmbeddings()

# Create a vector store using the created chuns and the embeddings model
vector = FAISS.from_documents(documents, embeddings)

# Create ChatOpenAI object for acting as an LLM.
# See https://python.langchain.com/docs/integrations/chat/openai/
llm = ChatOpenAI(api_key=os.environ["OPENAI_API_KEY"])

# Create a prompt template that gives the model a persona of a customer
# Create a chain for passing a list of Documents to a model.
output_parser = StrOutputParser()
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""", output_parser = output_parser)

document_chain = create_stuff_documents_chain(llm, prompt)
# document_chain = prompt | llm

# Creating a retriever
# See https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/vectorstore/
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)


CPU times: user 518 ms, sys: 63.2 ms, total: 581 ms
Wall time: 1.94 s


### Setup Tools and agents

In [17]:
@tool
def amazon_product_search(query: str):
    """Search for information about Amazon products.
    For any questions related to Amazon products, this tool must be used."""

    retriever_tool = create_retriever_tool(
        retriever,
        name="amazon_search",
        description="Search for information about Amazon products"
    )
    return retriever_tool.invoke(query)

@tool
def search_tavily(query: str):
    """
    Executes a web search using the TavilySearchResults tool.

    Parameters:
      query (str): The search query entered by the user.

    Returns:
      list: A list of search results containing answers, raw content, and images.
    """
    search_tool = TavilySearchResults(
        max_results=5,
        include_answer=True,  # include direct answers when available
        include_raw_content=True,
        include_images=True
    )

    return search_tool.invoke(query)

tools = [amazon_product_search, search_tavily]

# hwchase17/react is a prompt template designed for ReAct-style
# conversational agents.
prompt = hub.pull("hwchase17/react")

print(prompt.template)

llm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

react_agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt)

agent_executor = AgentExecutor(
    agent=react_agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)
    

/Users/ericmelz/Data/code/review-genie/venv/lib/python3.13/site-packages/langsmith/client.py:277: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(


Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [18]:
res = agent_executor.invoke({"input": "What is the best shoes I can find on Amazon?"})



> Entering new AgentExecutor chain...
To find the best shoes available on Amazon, I should search for popular and highly-rated options. I'll use the Amazon product search tool to gather relevant information.  
Action: amazon_product_search  
Action Input: "best shoes"  The Remora Climbing Shoe is Mad Rock's do-it-all slipper for climbers who can't have separate shoes for boulders, sport routes, and gyms. With a moderately stiff, slightly downturned design, the Remora performs on any climb at steep to vertical

steep to vertical angles. Science Friction rubber edges on jibs with ease, and it encapsulates the forefoot for toe-hooking prowess. The SynFlex upper conforms to your foot without much stretch.

Title
adidas Men's Predator 18+ FG Firm Ground Soccer Cleats
Description
adidas Predator 18+ FG- Black 7.5

Title
Kenneth Cole REACTION Men's Crespo Loafer B Shoe, Cognac, 10 M USI need to gather more options to provide a comprehensive answer about the best shoes available on Amazon. I

In [19]:
res = agent_executor.invoke({"input": "What is the current weather in Toronto?"})



> Entering new AgentExecutor chain...
I need to find the current weather information for Toronto. Since this is a general knowledge question, I will perform a web search to get the latest weather updates.  
Action: search_tavily  
Action Input: "current weather in Toronto"  [{'title': 'Toronto Weather Forecast: A Windy, Rainy Start to Spring!', 'url': 'https://www.netnewsledger.com/2025/03/20/toronto-weather-forecast-a-windy-rainy-start-to-spring/', 'content': 'Toronto is waking up to a damp and blustery start this Thursday, March 20, 2025. The temperature is currently sitting at 13.6°C, but don’t get too comfortable—it’s expected to take a nosedive by the afternoon. Light rain is falling over the city, with a southwest wind howling at 36 km/h, gusting even stronger as the day goes on. The barometric pressure sits at 99.7 kPa and is falling, which means unsettled weather ahead. Humidity is at 68%, making it feel a bit damp.', 'score': 0.95598555, 'raw_content': 'Toronto Weather Forec

In [21]:
res = agent_executor.invoke({"input":"what is the current stock price of SMCI?"})



> Entering new AgentExecutor chain...
I need to find the current stock price of SMCI (Super Micro Computer, Inc.). I'll perform a web search to get the latest stock price information.  
Action: search_tavily  
Action Input: "current stock price of SMCI"  [{'title': 'NASDAQ: SMCI Super Micro Computer Inc Stock - WallStreetZen', 'url': 'https://www.wallstreetzen.com/stocks/us/nasdaq/smci', 'content': "(NASDAQ: SMCI) Super Micro Computer's market cap is $22.49B, as of Mar 20, 2025. ... SMCI's current stock price of $37.90 by SMCI's total outstanding shares of", 'score': 0.9291904}, {'title': 'Super Micro Computer (NASDAQ:SMCI) - Share Price', 'url': 'https://www.intelligentinvestor.com.au/shares/nasdaq-smci/super-micro-computer/share-price', 'content': "Super Micro Computer's current share price is $40.10. This constitutes a share price movement of -5.58% when compared to its closing", 'score': 0.9090688715083798}, {'title': 'Stock Information - Stock Details - Super Micro Computer, Inc

In [22]:
res = agent_executor.invoke({"input": "How to install Backsplash Wallpaper? Also find some brands on Amazon"})



> Entering new AgentExecutor chain...
To answer the question about installing backsplash wallpaper and to find some brands available on Amazon, I will first search for installation instructions and tips. Then, I will look for brands of backsplash wallpaper on Amazon.

Action: search_tavily  
Action Input: "how to install backsplash wallpaper"  [{'title': 'How to install a Removable Wallpaper Backsplash', 'url': 'https://www.delineateyourdwelling.com/how-to-install-a-removable-wallpaper-backsplash/', 'content': "- Install the wallpaper from the top of your backsplash to the bottom. This ensures that your patterns will match up correctly and you have manageable sized sections to adhere to the wall. - If you can't seem to get a few bubbles here and there out with the wallpaper's smoothing tool, you can use a small needle pin and pop the bubbles.", 'score': 0.8591094}, {'title': 'How to Wallpaper a Backsplash - The Homes I Have Made', 'url': 'https://thehomesihavemade.com/how-to-wallpape

### Agent with Chat History

In [27]:
session_memory = {}
session_id = "my-session"

def get_memory(session_id):
    """Fetch or create a chat history instance for a given session."""
    if session_id not in session_memory:
        session_memory[session_id] = ChatMessageHistory()
    return session_memory[session_id]

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: get_memory(session_id),
    input_messages_key="input",
    history_messages_key="chat_history"
)

def chat_with_agent(user_input, session_id):
    """Processes user input and maintains session-based chat history."""
    memory = get_memory(session_id)
    #memory.clear()
    response = agent_with_chat_history.invoke(
        {"input": user_input, "chat_history": memory.messages},
        config={"configurable": {"session_id": session_id}}
    )

    if isinstance(response, dict) and "output" in response:
        return response["output"]
    else:
        return "Error: Unexpected response format"

In [28]:
chat_with_agent("What is the latest fashion trends in 2025?", session_id)



> Entering new AgentExecutor chain...
To find the latest fashion trends in 2025, I should perform a web search to gather the most current information and insights on this topic. 

Action: search_tavily  
Action Input: "latest fashion trends 2025"  [{'title': '2025 Fashion Trends: Top 10 Forecasted Key Trends - Heuritech', 'url': 'https://heuritech.com/fashion-trends-2025/', 'content': 'Strapless dresses made a striking comeback on the Spring Summer 2025 runways, blending minimalist charm with bold new textures. Acne Studios showcased a heart-shaped neckline that created an exaggerated silhouette, adding a playful twist to classic black. MM6 Maison Margiela embraced a deconstructed approach, with ruched details giving an edgy twist to the strapless form. At LaQuan Smith, snake-print strapless designs lent a sultry allure, highlighting the trend’s versatile appeal. [...] Top fashion trends for 2025: A look into the Future\nWith a combination of predicted data from Heuritech for Spring/

'The latest fashion trends for 2025 include boho chic in suede, aquatic influences, soft power workwear, nostalgia-inspired styles, bold colors and prints, elevated sportswear, excess and glamour, and asymmetrical designs.'

In [29]:
chat_with_agent("Does amazon have these latest trends?", session_id)



> Entering new AgentExecutor chain...
I need to find out what the latest trends are and then check if Amazon has products related to those trends. I'll start by searching for the latest trends.  
Action: search_tavily  
Action Input: "latest trends 2023"  [{'title': 'The 5 Biggest Business Trends In 2023 Everyone Must Get Ready ... - Forbes', 'url': 'https://www.forbes.com/sites/bernardmarr/2022/10/03/the-5-biggest-business-trends-for-2023/', 'content': 'In 2023, we see the continuation of innovations and developments in transformative technologies such as artificial intelligence (AI), the internet of things (IoT), virtual and augmented reality (VR/AR), cloud computing, blockchain, and super-fast network protocols like 5G. What’s more, these transformational digital technologies do not exist in isolation from each other, and we will see the boundaries between them blurring. New solutions for augmented working, hybrid and remote working,', 'score': 0.8088424229172142, 'raw_content': '

'Agent stopped due to iteration limit or time limit.'

### UI

In [30]:
# Create Gradio app interface
with gr.Blocks() as app:
    gr.Markdown("# 🤖 Review Genie - Agents & ReAct Framework")
    gr.Markdown("Enter your query below and get AI-powered responses with session memory.")

    with gr.Row():
        input_box = gr.Textbox(label="Enter your query:", placeholder="Ask something...")
        output_box = gr.Textbox(label="Response:", lines=10)

    submit_button = gr.Button("Submit")

    submit_button.click(chat_with_agent, inputs=input_box, outputs=output_box)

# Launch the Gradio app
app.launch(debug=True, share=True)


/Users/ericmelz/Data/code/review-genie/venv/lib/python3.13/site-packages/gradio/utils.py:1024: UserWarning: Expected 2 arguments for function <function chat_with_agent at 0x10f8ca7a0>, received 1.
  warnings.warn(
/Users/ericmelz/Data/code/review-genie/venv/lib/python3.13/site-packages/gradio/utils.py:1028: UserWarning: Expected at least 2 arguments for function <function chat_with_agent at 0x10f8ca7a0>, received 1.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://8f14850ba7d5f228f6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


/Users/ericmelz/Data/code/review-genie/venv/lib/python3.13/site-packages/gradio/helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")




> Entering new AgentExecutor chain...
To find the latest fashion trends in 2025, I should perform a web search to gather the most current information and insights on this topic. 

Action: search_tavily  
Action Input: "latest fashion trends 2025"  [{'title': '2025 Fashion Trends: Top 10 Forecasted Key Trends - Heuritech', 'url': 'https://heuritech.com/fashion-trends-2025/', 'content': 'Strapless dresses made a striking comeback on the Spring Summer 2025 runways, blending minimalist charm with bold new textures. Acne Studios showcased a heart-shaped neckline that created an exaggerated silhouette, adding a playful twist to classic black. MM6 Maison Margiela embraced a deconstructed approach, with ruched details giving an edgy twist to the strapless form. At LaQuan Smith, snake-print strapless designs lent a sultry allure, highlighting the trend’s versatile appeal. [...] Top fashion trends for 2025: A look into the Future\nWith a combination of predicted data from Heuritech for Spring/

/Users/ericmelz/Data/code/review-genie/venv/lib/python3.13/site-packages/gradio/helpers.py:968: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")




> Entering new AgentExecutor chain...
To determine if Amazon has the latest trends, I should search for current trending products on Amazon. This will help me find out what the latest trends are and if they are available on the platform. 

Action: amazon_product_search  
Action Input: "latest trends"  Title
Stunned Trendy, Latest, Classy Attractive Look Printed Mobile Back Case Cover for Vivo Y73 (Heart Lock)
Description

trade routes to the East. The timeless beauty of an antique leather binding is brought into the present on the cover of this deep blue book.

from head to toe (barefoot)<br /><br />1. Originally Designed: All dresses are designed by the most qualified designers, and the most experienced tailors carry out strict quality inspections in our own factories. Every needle and every thread

or vegetables fresh.I need to refine my search to find more specific information about the latest trending products on Amazon, as the initial results were not very relevant. 

Action: am